In [181]:
import sys
sys.path.append('..')

import numpy as np
import socnet as sn

In [182]:
sn.graph_width = 500
sn.graph_height = 500
sn.node_size = 10
sn.edge_width = 1
sn.edge_color = (192, 192, 192)
sn.node_label_position = 'top center'

In [183]:
print(dir(sn))

['HEAD_ANGLE', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_add_edge', '_add_node', '_build_edge_trace', '_build_label_trace', '_build_layout', '_build_node_trace', '_convert', '_normalize_positions', '_rotate', '_scale', '_set_layout', 'build_betweenness', 'build_matrix', 'build_shortest_paths', 'cos', 'edge_color', 'edge_label_distance', 'edge_width', 'generate_complete_graph', 'generate_empty_graph', 'generate_frame', 'graph_height', 'graph_width', 'load_graph', 'networkx', 'node_color', 'node_label_position', 'node_size', 'plotly', 'random', 'randomize_positions', 'reset_edge_colors', 'reset_node_colors', 'reset_positions', 'show_animation', 'show_graph', 'sin', 'sqrt', 'update_positions']


In [233]:
total_nodes = 60
percentage_women = 0.2
percentage_man = 1 - percentage_women

constant_prejudice_women = 0.8

g = sn.generate_empty_graph(total_nodes)
# g = g.to_directed()

In [234]:
from random import randint

for n in range(int(total_nodes * percentage_man)):
    g.node[n]['gender'] = 'male'
    g.node[n]['ability'] = randint(0, 10)
    g.node[n]['color'] = (255,255,0)
    
for n in range(int(total_nodes * percentage_man), total_nodes):
    g.node[n]['gender'] = 'female'
    g.node[n]['ability'] = randint(0, 6)
    g.node[n]['color'] = (255,0,255)


In [235]:
for n in g.nodes():
    for i in g.nodes():
        if n != i:
            ability_n = g.node[n]['ability']
            ability_i = g.node[i]['ability']
            if g.node[n]['gender'] == 'male' and g.node[i]['gender'] == 'female':
                ability_i = max(ability_i - 1,0)
            if ability_i >= ability_n and randint(0,10) == 10:
                g.add_edge(n,i)

In [236]:
sn.reset_edge_colors(g)
sn.show_graph(g)

In [237]:
from scipy.stats.stats import pearsonr
from scipy.optimize import minimize

def cost(c, A, mask):
    row = np.matrix(c)

    C = np.multiply(row.transpose(), row)

    correlation, _ = pearsonr(C[~mask].flat, A[~mask].flat)

    return -correlation

def calculate_coreness(g):
    length = g.number_of_nodes()

    c = np.random.rand(length)

    A = sn.build_matrix(g)

    mask = np.eye(length, dtype=bool)

    bounds = [(0, 1) for _ in range(length)]

    result = minimize(cost, c, args=(A, mask), bounds=bounds)

    norm = 0

    for n in g.nodes:
        g.nodes[n]['coreness'] = result['x'][n]

        norm += g.nodes[n]['coreness'] ** 2

    for n in g.nodes:
        g.nodes[n]['coreness'] /= norm

In [238]:
calculate_coreness(g)

In [239]:
for n in g.nodes():
    print(n, g.node[n]['gender'], g.node[n]['coreness'])

0 male 0.0
1 male 0.0383017330901
2 male 0.0
3 male 0.0
4 male 0.0401811028092
5 male 0.0
6 male 0.0399674895908
7 male 0.0
8 male 0.0
9 male 0.0
10 male 0.0
11 male 0.0
12 male 0.0
13 male 0.0383440534761
14 male 0.0
15 male 0.0
16 male 0.0
17 male 0.0
18 male 0.0
19 male 0.0
20 male 0.0
21 male 0.0400415876339
22 male 0.0
23 male 0.0
24 male 0.0
25 male 0.0365067293175
26 male 0.0
27 male 0.0
28 male 0.0
29 male 0.0400768359722
30 male 0.979825661567
31 male 0.0
32 male 0.0
33 male 0.0
34 male 0.00253172271429
35 male 0.0
36 male 0.0
37 male 0.0
38 male 0.0399619108948
39 male 0.0
40 male 0.0
41 male 0.0
42 male 0.0
43 male 0.0
44 male 0.0
45 male 0.0
46 male 0.0366507728444
47 male 0.0
48 female 0.0
49 female 0.000873412197371
50 female 0.0
51 female 0.0
52 female 0.0416371117231
53 female 0.0
54 female 0.0
55 female 0.0399716886041
56 female 0.0381645381641
57 female 0.0366139095913
58 female 0.0
59 female 0.0
